In [1]:
import matplotlib.pyplot as plt
import transformers
import pandas as pd
import numpy as np
import torch
import json
import sys
import os

# from dotenv import load_dotenv
from huggingface_hub import login

In [3]:
# load_dotenv()
# path = os.environ['DATA_PATH']
# os.environ["ACCELERATE_USE_TORCH_DEVICE"] = "true"
path = os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", "data", "ita")
hf_token_path = os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", "HF_TOKEN.txt")
with open(hf_token_path, "r") as f:
    hf_token = f.read().strip()

login(token=hf_token)
cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

In [4]:
device = torch.device("cuda")
print(torch.cuda.get_device_name(0))
print("Supports float16:", torch.cuda.is_available())
print("Supports bfloat16:", torch.cuda.is_bf16_supported())

NVIDIA GeForce RTX 3050
Supports float16: True
Supports bfloat16: True


In [5]:
datasetRaw = {}

for i in [ocred_fn, cleaned_fn]:
    if i not in os.listdir(path):
        print(f"ERROR 404 ! File {i} not Found...")

    file_path = os.path.join(path, i)
    with open(file_path, 'r') as f:
        datasetRaw[i.split('.')[0]] = json.load(f)
        f.close()

In [6]:
print(datasetRaw['cleaned']['1'])

I.

Come andò che Maestro Ciliegia, falegname
trovò un pezzo di legno che piangeva e rideva come un bambino.

— C’era una volta....
— Un re! — diranno subito i miei piccoli lettori.
— No, ragazzi, avete sbagliato. C’era una volta un pezzo di legno.
Non era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d’inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze.
Non so come andasse, ma il fatto gli è che un bel giorno questo pezzo di legno capitò nella bottega di un vecchio falegname, il quale aveva nome mastr’Antonio, se non che tutti lo chiamavano maestro Ciliegia, per via della punta del <img description>.... sentì una vocina sottile sottile. suo naso, che era sempre lustra e paonazza, come una ciliegia matura.
Appena maestro Ciliegia ebbe visto quel pezzo di legno, si rallegrò tutto; e dandosi una fregatina di mani per la contentezza, borbottò a mezza voce:
— Questo legno è capitato a tempo; voglio servirmene per fare una ga

In [7]:
def number_words(sentence):
    sentence = sentence.split()
    return len(sentence)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# === 1. Carica modello Minerva ===
model_name = "sapienzanlp/Minerva-3B-base-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map=device, torch_dtype=torch.float16)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [11]:
import re

def smart_split_paragraphs(text):
    # 1. Normalizza spazi doppi
    text = re.sub(r'\s+', ' ', text.strip())

    # 2. Proteggi le sequenze di puntini ("...") da split
    text = re.sub(r'\.{2,}', lambda m: f"<DOTS{len(m.group(0))}>", text)

    # 3. Split solo su punto singolo seguito da spazio e mai su DOTS
    fragments = re.split(r'(?<!\.)\.(?!\.)(?=\s|$)', text)

    result = []
    for frag in fragments:
        frag = frag.strip()
        if not frag:
            continue
        # Ripristina puntini
        frag = re.sub(r'<DOTS(\d+)>', lambda m: '.' * int(m.group(1)), frag)
        result.append(frag + ". ")  # Rimettiamo il punto

    return result

def merge_hyphenated_words(text):
    # Casi classici: parola spezzata con trattino seguito da newline o spazio
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    return text

sample = datasetRaw['original_ocr']['1']
sentences = smart_split_paragraphs(sample)

for i in range(len(sentences)):
    sentences[i] = merge_hyphenated_words(sentences[i])

for i, s in enumerate(sentences, 1):
    print(f"{s}")

I. 
Como andò che Maestro Ciliegia, Megnamc trovò un pezzo di legno che piangeva e rideva come un bambino. 
— C'era una volta.... — Un re! -diranno subito i miei piccoli lettori. 
— Ko, ragazzi, avete sbagliato. 
C'era una volta un pezzo di legno. 
Kon era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d' inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze. 
Non so come andasse, ma il fatto gli è che un bel giorno' questo pezzo di legno capitò nella bottega di un vecchio falegname, il quale aveva nome mastr' Antonio, se non che tutti lo chiamavano maestro Ciliegia, per via della punta del sentì nna vocina sottile sottile. 
SUO naso, che era sempre lustra e paonazza, come una ciliegia matura. 
Appena maestro Ciliegia ebbe visto quel pezzo di legno, si rallegrò tutto; e dandosi una fregatina di mani per la contentezza, borbottò a mezza voce: — Questo legno è capitato a tempo; voglio servirmene per fare una gamba di tavolino.

In [ ]:
def correct_with_minerva(texts):
    results = []
    for text in texts:
        # text_fixed = merge_hyphenated_words(text)

        # prompt = (
        #     f"Correggi SOLO gli OCR nel testo qui sotto estratto dal libro 'Pinocchio' di Collodi. Analizza ogni parola attentamente per individuare e risolvere solo gli errori di battitura, frammentazione o capitalizzazione." # <-- AGGIUNTA
        #     f"NON modificare il significato, aggiungere o togliere parole, o alterare la punteggiatura. "
        #     f"Mantieni il numero di parole e l'ordine delle parole fedelmente. "
        #     f"La tua unica funzione è la correzione di errori OCR: lettere errate, spezzature, e **ERRORI DI CAPITALIZZAZIONE**. "
        #     f"Rendi maiuscole le iniziali di frase e i nomi propri. Rendi minuscole le parole che non sono nomi propri o inizi di frase ma sono in maiuscolo. "
        #     f"Esempi di correzione OCR (inclusa capitalizzazione):\n"
        #     f"Input: 'Qvando il sole sorge, le ombrre svannos noel silenzio del matino. Kon era un legno di lusso. SUO naso, che era sempre lustra.'\n"
        #     f"Output: 'Quando il sole sorge, le ombre svaniscono nel silenzio del mattino. Non era un legno di lusso. Suo naso, che era sempre lustra.'\n"
        #     f"Testo OCR: {text_fixed} "
        #     f"Testo corretto:"
        # )
        
        prompt = (
            f"Sei un generatore di eBooks. "
            f"Correggi TUTTI gli errori OCR nel testo che ti verrà fornito. "
            f"Non aggiungere, togliere o cambiare parole. "
            f"Non modificare la punteggiatura esistente. "
            f"Non interpretare, riassumere o riscrivere il testo. "
            f"Mantieni esattamente lo stesso numero di parole e l'ordine delle parole. "
            f"Il testo corretto deve conservare il significato originale inalterato. "
            f"La tua unica funzione è la correzione di errori OCR: lettere errate, parole spezzate, ed errori di capitalizzazioni. "
            f"Rendi maiuscole le iniziali di frase e i nomi propri. Rendi minuscole le parole che non sono nomi propri o inizi di frase ma sono in maiuscolo. "
            f"Di seguito sono forniti esempi di input OCR e il corrispondente output corretto. Apprendi da questi esempi per svolgere il tuo compito.\n"
            f"Esempi di correzione OCR (inclusa capitalizzazione):\n"
            f"Input:  'qvesto è un testo. sembra ocr.'\n"
            f"Output: 'Questo è un testo. Sembra OCR.'\n"
            f"Input:  'l'aquila vola alta. il cieto è blù.'\n"
            f"Output: 'L'aquila vola alta. Il cielo è blu.'\n"
            f"Input: 'la matita è rota. c'è del tem po.'\n"
            f"Output: 'La matita è rotta. C'è del tempo.'\n"
            f"Input: 'i1 libro è su1 tavolo. LA STAMPA è chiara.'\n"
            f"Output: 'Il libro è sul tavolo. La stampa è chiara.'\n"
            f"Input: 'abbiam o un gTande paeco. la vitta è bella.'\n"
            f"Output: 'Abbiamo un grande pacco. La vita è bella.'\n"
            # f"Input: 'rn'\n"
            # f"Output: 'm'\n"
            # f"Input: 'Ko'\n"
            # f"Output: 'No'\n"
            # f"Input: 'Ko'\n"
            # f"Output: 'No'\n"
            # f"Input: '1'\n"
            # f"Output: 'i'\n"
            # f"Input: '0'\n"
            # f"Output: 'o'\n" 
            # f"Input: 'nn'\n"
            # f"Output: 'un'\n"
            # f"Input: 'gT'\n"
            # f"Output: 'gr'\n"
            # f"Input: 'cl'\n"
            # f"Output: 'd'\n"
            # f"Input: 'e, '\n"
            # f"Output: 'e '\n"
            f"Testo OCR: {text} "
            f"Testo corretto:"
        )
        
        output = generator(
            prompt,
            # min_new_tokens=len(tokenizer.encode(text_fixed)) - 2,
            max_new_tokens=len(tokenizer.encode(text)) - 2,
            do_sample=True,
            temperature=0.3,
            top_p=1.0,
            num_beams=4,
            repetition_penalty=1.1,
        )[0]["generated_text"]

        cleaned = output.split("Testo corretto:")[-1].strip().split("\n")[0]

        print(f"Input:  {text}")
        print(f"Output: {cleaned}\n")
        
        results.append({"input": text, "output": cleaned})

    return results

# # === 4. Test ===
# ocr_text = ["Qvando il sole sorge, le ombrre svannos noel silenzio del matino.", "se non che tutti lo chiamavano maestro Ciliegia, per via della punta del sentì nna vocina sottile sottile."]
# corrected_data = correct_with_minerva(sentences[1:7])
# 
# # === 5. Stampa risultato ===
# for clean in corrected_data:
#     print(f"Input:  {clean['input']}")
#     print(f"Output: {clean['output']}\n")
#     
# # with open(os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", "ocr_corretti.json"), "a", encoding="utf-8") as f:
# #     json.dump(corrected_data, f, ensure_ascii=False)

In [ ]:
# rivedere cap 2
for i in range(1, 8):
    print(f"Paragraph {i}")
    chapter = i
    sample = datasetRaw['original_ocr'][str(chapter)]
    sentences = smart_split_paragraphs(sample)
    for i in range(len(sentences)):
        sentences[i] = merge_hyphenated_words(sentences[i])

    corrected_data = correct_with_minerva(sentences)

    with open(os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", f"Chapter_{chapter}.json"), "a", encoding="utf-8") as f:
        json.dump(corrected_data, f, ensure_ascii=False)

Paragraph 1
Input:  I. 
Output: I.

Input:  Como andò che Maestro Ciliegia, Megnamc trovò un pezzo di legno che piangeva e rideva come un bambino. 
Output: Come andò che Maestro Ciliegia, Megnamc trovò un pezzo di legno che piangeva e rideva come un bambino.

Input:  — C'era una volta.... — Un re! -diranno subito i miei piccoli lettori. 
Output: — C'era una volta.... — Un re! —diranno subito i miei piccoli lettori.

Input:  — Ko, ragazzi, avete sbagliato. 
Output: — Ko, ragazzi, avete sbagliato.

Input:  C'era una volta un pezzo di legno. 
Output: C'era una volta un pezzo di legno.

Input:  Kon era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d' inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze. 
Output: Kon era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d' inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze.

Input:  Non so come andasse, ma il fatto g

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [9]:
model_name = "meta-llama/Llama-3.2-3B"

In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

llama_tokenizer = AutoTokenizer.from_pretrained(model_name)
llama = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", 
    torch_dtype="auto"
)

llama_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [59]:
def correct_with_llama(texts):
    results = []
    for text in texts:

        # Prompt dettagliato con esempi specifici per errori OCR (capitalizzazione, numeri, lettere)
        prompt = (
            f"Sei un Grillo Parlante! "
            f"Correggi TUTTI gli errori OCR nel testo che ti verrà fornito. "
            f"Non aggiungere, togliere o cambiare parole. "
            f"Non modificare la punteggiatura esistente. "
            f"Non interpretare, riassumere o riscrivere il testo. "
            f"Mantieni esattamente lo stesso numero di parole e l'ordine delle parole. "
            f"Il testo corretto deve conservare il significato originale inalterato. "
            f"La tua unica funzione è la correzione di errori OCR: lettere errate, parole spezzate, ed errori di capitalizzazioni. "
            f"Rendi maiuscole le iniziali di frase e i nomi propri. Rendi minuscole le parole che non sono nomi propri o inizi di frase ma sono in maiuscolo. "
            f"Di seguito sono forniti esempi di input OCR e il corrispondente output corretto. Apprendi da questi esempi per svolgere il tuo compito.\n"
            f"Esempi di correzione OCR (inclusa capitalizzazione):\n"
            f"Input: 'qvesto è un testo. sembra ocr.'\n"
            f"Output: 'Questo è un testo. Sembra OCR.'\n"
            f"Input: 'l'aquila vola alta. il cieto è blù.'\n"
            f"Output: 'L'aquila vola alta. Il cielo è blu.'\n"
            f"Input: 'la matita è rota. c'è del tem po.'\n"
            f"Output: 'La matita è rotta. C'è del tempo.'\n"
            f"Input: 'i1 libro è su1 tavolo. LA STAMPA è chiara.'\n"
            f"Output: 'Il libro è sul tavolo. La stampa è chiara.'\n"
            f"Input: 'abbiam o un gTande paeco. la vitta è bella.'\n"
            f"Output: 'Abbiamo un grande pacco. La vita è bella.'\n"
            f"Input: 'rn'\n"
            f"Output: 'm'\n"
            f"Input: 'Ko'\n"
            f"Output: 'No'\n"
            f"Input: '1ibro'\n" # Esempio per numeri scambiati per lettere
            f"Output: 'libro'\n"
            f"Input: '0cchio'\n" # Esempio per numeri scambiati per lettere
            f"Output: 'occhio'\n"
            f"Input: 'cl'\n" # Esempio di ambiguità tipica OCR
            f"Output: 'd'\n"
            f"Input: 'e, '\n" # Esempio di punteggiatura errata
            f"Output: 'e '\n"
            f"Testo OCR: {text} " # Usa il testo pre-elaborato nel prompt
            f"Testo corretto:"
        )

        # Calcola la lunghezza del prompt e del testo corretto in token per i parametri di generazione
        expected_output_tokens_length = len(llama_tokenizer.encode(text)) # Usa il testo già corretto dai trattini

        # Generazione del testo
        # È fondamentale passare il prompt come una lista al generatore della pipeline,
        # anche se è una singola stringa. Questo risolve il TypeError precedente.
        output = llama_generator(
            prompt,
            max_new_tokens=expected_output_tokens_length, # Margine maggiore per sicurezza
            do_sample=True,
            temperature=0.01,
            top_p=1.0,
            num_beams=4,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )[0]["generated_text"]

        # Estrazione e pulizia dell'output
        try:
            cleaned = output.split("Testo corretto:")[-1].strip()
            cleaned = cleaned.split('\n')[0].strip()
        except IndexError:
            cleaned = output.strip()
            print(f"ATTENZIONE: 'Testo corretto:' non trovato nell'output. Output grezzo: {output[:100]}...")
        
        # Gestione di casi limite dove l'output potrebbe essere vuoto dopo lo split
        if not cleaned and len(output.split("Testo corretto:")) > 1:
            # Se è vuoto ma c'è stato un split, prova a recuperare qualcosa dalla riga successiva o fallback
            potential_next_line = output.split("Testo corretto:")[-1].split('\n', 1)
            if len(potential_next_line) > 1:
                cleaned = potential_next_line[1].strip().split('\n')[0].strip()
            else:
                cleaned = output.strip()

        print(f"Input: \t{text}")
        print(f"Output:\t{cleaned}\n")

        results.append({"input": text, "output": cleaned})

    return results

In [60]:
for i in range(1, 8):
    print(f"Paragraph {i}")
    chapter = i
    sample = datasetRaw['original_ocr'][str(chapter)]
    sentences = smart_split_paragraphs(sample)
    for i in range(len(sentences)):
        sentences[i] = merge_hyphenated_words(sentences[i])

    corrected_data = correct_with_llama(sentences)

    with open(os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", f"Chapter_{chapter}.json"), "a", encoding="utf-8") as f:
        json.dump(corrected_data, f, ensure_ascii=False)

Paragraph 1
Input: 	I. 
Output:	I.

Input: 	Como andò che Maestro Ciliegia, Megnamc trovò un pezzo di legno che piangeva e rideva come un bambino. 
Output:	Come andò che Maestro Ciliegia, Megnamc trovò un pezzo di legno che piangeva e rideva come un bambino.

Input: 	— C'era una volta.... — Un re! -diranno subito i miei piccoli lettori. 
Output:	— C'era una volta.... — Un re! -diranno subito i miei piccoli lettori.

Input: 	— Ko, ragazzi, avete sbagliato. 
Output:	— No, ragazzi, avete sbagliato.

Input: 	C'era una volta un pezzo di legno. 
Output:	C'era una volta un pezzo di legno.

Input: 	Kon era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d' inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze. 
Output:	Con era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d'inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze.

Input: 	Non so come andasse, ma il fatto gl